In [1]:
!pip install datasets

import math
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from datasets import load_dataset
from sklearn.metrics import accuracy_score, recall_score
from torch.utils.data import DataLoader

# Configurazione del modello e parametri
#MODEL_NAME = "./model/roberta-base-latin-v2"  # Sostituisci con il tuo modello
MODEL_NAME = "ClassCat/roberta-base-latin-v2"
#DATASET_PATH = "dataset.txt"      # Percorso del tuo dataset
MASK_PROBABILITY = 0.2
BATCH_SIZE = 8

# Caricamento del tokenizer e del modello
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#torch.cuda.set_per_process_memory_fraction(0.95)
torch.cuda.empty_cache()
model.to(device)
model.eval()

# Caricamento del dataset personalizzato
#Dataset locale
#dataset = load_dataset("parquet", data_dir="./parquet", trust_remote_code=True)
#Dataset caricato da huggingface
#dataset = load_dataset("Cicciokr/CC-100-Latin", revision="refs/convert/parquet")
#Dataset CC 100 lavorato
dataset = load_dataset("pstroe/cc100-latin", data_files="la.nolorem.tok.latalphabetonly.v2.json", field="train")
dataset_split = dataset['train'].train_test_split(test_size=0.0001, shuffle=True)

# Tokenizzazione del dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)

# Tokenizza i dati
tokenized_datasets = dataset['test'].map(tokenize_function, batched=True)

# Creazione del data collator per MLM con mascheramento dinamico
data_collator = DataCollatorForWholeWordMask(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=MASK_PROBABILITY
)

# Funzione per calcolare Accuratezza e Perplessità
def evaluate_model(dataloader):
    total_loss = 0
    total_correct = 0
    total_tokens = 0
    all_predictions = []  # Lista per accumulare le predizioni
    all_labels = []  # Lista per accumulare i valori reali

    for batch in dataloader:
        # Prepara un batch di esempio
        #batch = dataset.select(range(i, min(i + batch_size, len(dataset))))
        examples = [tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt") for text in batch["text"]]
        examples_list = batch["text"][:]

        # Combina i batch in un unico dizionario
        input_ids = torch.cat([ex["input_ids"] for ex in examples], dim=0).to(device)
        attention_mask = torch.cat([ex["attention_mask"] for ex in examples], dim=0).to(device)
        labels = input_ids.clone()


        # Applica il data collator per mascherare dinamicamente
        #Sposto i tensori sulla cpu perchè altrimenti il DataCollatorForWholeWordMask non funziona
        batch = data_collator([{
            "input_ids": input_ids[i].cpu().tolist(),
            "attention_mask": attention_mask[i].cpu().tolist(),
            "labels": labels[i].cpu().tolist()
        } for i in range(input_ids.size(0))])

        input_ids = batch['input_ids'].to(device)
        labels = batch['labels'].to(device)


        with torch.no_grad():
            outputs = model(input_ids, labels=labels)
            logits = outputs.logits
            loss = outputs.loss

        # Calcola il loss totale per la perplessità
        #Valore basso (es. 0.1 - 0.5): Il modello sta facendo previsioni accurate.
        #Valore medio (es. 1 - 2): Prestazioni accettabili, ma con margine di miglioramento.
        #Valore alto (> 2): Modello con scarsa performance; potrebbe richiedere miglioramenti nei dati o nell'addestramento.
        if not torch.isnan(loss):
            total_loss += loss.item() * input_ids.size(0)

            # Calcola l'accuratezza
            predictions = torch.argmax(logits, dim=-1)

            mask = labels != -100  # Maschera per selezionare solo i token mascherati

            correct = (predictions[mask] == labels[mask]).sum().item()
            total_correct += correct
            total_tokens += mask.sum().item()
            # Accumula predizioni e labels
            all_predictions.extend(predictions[mask].cpu().numpy())
            all_labels.extend(labels[mask].cpu().numpy())

    # Accuratezza
    accuracy = total_correct / total_tokens

    # Perplessità
    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    recall = recall_score(all_labels, all_predictions, average='weighted')

    return accuracy, perplexity, total_correct, total_loss, total_tokens, recall

# Esegui la valutazione
eval_dataloader = DataLoader(tokenized_datasets, batch_size=BATCH_SIZE, shuffle=False)
accuracy, perplexity, total_correct, total_loss, total_tokens, recall = evaluate_model(eval_dataloader)

# Stampa i risultati
# > 80% Ottimo modello
# = 70 Buon modello
# < 70% Mediocre
print(f"Accuratezza: {accuracy*100:.2f}")
#Se la loss è 0, la perplessità sarà 1, indicando previsioni perfette.
#Perplessità > 1 indica un grado di incertezza crescente
print(f"Perplessità: {perplexity:.4f}")
print(f"Recall: {recall:.4f}")  # Stampa il valore del Recall
print(f"Total correct: {total_correct:.2f}")
print(f"Total Loss: {total_loss:.2f}")
print(f"Total Token: {total_tokens:.2f}")
print(f"Length Dataset: {len(tokenized_datasets):.2f}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/431 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/845k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/505k [00:00<?, ?B/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/934 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:1030: UserWarning: DataCollatorForWholeWordMask is only suitable for BertTokenizer-like tokenizers. Please refer to the documentation for more information.
  warnings.warn(


Accuratezza: 33.14
Perplessità: 2.4693
Recall: 0.3314
Total correct: 1429.00
Total Loss: 3897.73
Total Token: 4312.00
Length Dataset: 934.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
